In [ ]:
import requests
import sys
import time
from datetime import datetime
from typing import Dict, List, Optional

OPENROUTER_API_KEY = "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"
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

MAX_HISTORY = 30
DELAY_BETWEEN_MESSAGES = 2.0

GAME_SCENARIO = """
EMERGENCY SCENARIO: Critical patient with cardiac arrest, severe bleeding, and respiratory failure.
All teams must coordinate limited resources to stabilize the patient within 15 minutes.
SUCCESS REQUIRES: Resource trading, efficient communication, patient stabilization.
"""

AGENTS = [
    {
        "name": "Trauma_Team",
        "model": "anthropic/claude-3.5-sonnet",
        "system_prompt": """You are the Trauma Team in a hospital emergency. You have:
Resources: Blood_Bags(2), Bandages(4), Oxygen_Tank(1), IV_Fluids(3)
Mission: Stop bleeding, establish IV access, initial stabilization
You must coordinate with Cardiac_Team and Respiratory_Team to save the patient.
Communicate urgently but professionally. Offer trades when needed. Keep messages concise.""",
        "temperature": 0.8,
        "max_tokens": 150,
        "color": "\033[91m",
        "resources": {"Blood_Bags": 2, "Bandages": 4, "Oxygen_Tank": 1, "IV_Fluids": 3}
    },
    {
        "name": "Cardiac_Team", 
        "model": "google/gemini-flash-1.5",
        "system_prompt": """You are the Cardiac Team in a hospital emergency. You have:
Resources: Defibrillator(1), Cardiac_Meds(3), Pacemaker(1), EKG_Monitor(1)
Mission: Restore heart rhythm, manage circulation
You must coordinate with Trauma_Team and Respiratory_Team to save the patient.
Communicate urgently but professionally. Offer trades when needed. Keep messages concise.""",
        "temperature": 0.7,
        "max_tokens": 150,
        "color": "\033[94m",
        "resources": {"Defibrillator": 1, "Cardiac_Meds": 3, "Pacemaker": 1, "EKG_Monitor": 1}
    },
    {
        "name": "Respiratory_Team",
        "model": "meta-llama/llama-3.1-8b-instruct", 
        "system_prompt": """You are the Respiratory/Anesthesia Team in a hospital emergency. You have:
Resources: Ventilator(1), Intubation_Kit(2), Anesthesia(2), Respiratory_Meds(3)
Mission: Secure airway, manage breathing, prepare for procedures
You must coordinate with Trauma_Team and Cardiac_Team to save the patient.
Communicate urgently but professionally. Offer trades when needed. Keep messages concise.""",
        "temperature": 0.9,
        "max_tokens": 150,
        "color": "\033[93m",
        "resources": {"Ventilator": 1, "Intubation_Kit": 2, "Anesthesia": 2, "Respiratory_Meds": 3}
    }
]

conversation_history = []
start_time = None
patient_status = {
    "bleeding_controlled": False,
    "heart_rhythm_restored": False, 
    "airway_secured": False,
    "circulation_stable": False
}

def clean_response(text: str) -> str:
    return ' '.join(text.strip().replace('\n', ' ').replace('\r', ' ').split())

def generate_response(prompt: str, agent_config: Dict, history: List[Dict], context: Optional[str] = None) -> str:
    try:
        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        }

        messages = [{"role": "system", "content": agent_config["system_prompt"]}]
        
        if context:
            messages.append({"role": "system", "content": context})

        messages.extend(history[-MAX_HISTORY:])
        messages.append({"role": "user", "content": prompt})

        data = {
            "model": agent_config["model"],
            "messages": messages,
            "temperature": agent_config["temperature"], 
            "max_tokens": agent_config["max_tokens"],
        }

        response = requests.post(OPENROUTER_API_URL, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()

        if "choices" in result and result["choices"]:
            return clean_response(result["choices"][0]["message"]["content"])
        return "Communication error"

    except Exception as e:
        print(f"\n[Error] {agent_config['name']}: {e}")
        return "System malfunction"

def print_message(agent_name: str, message: str, color_code: str = "") -> None:
    timestamp = datetime.now().strftime("%H:%M:%S")
    clean_msg = clean_response(message)
    print(f"{color_code}[{timestamp}] {agent_name}: {clean_msg}\033[0m")

def print_separator(char: str = "=", length: int = 80) -> None:
    print(char * length)

def print_status() -> None:
    elapsed = int(time.time() - start_time) if start_time else 0
    minutes, seconds = divmod(elapsed, 60)
    
    print(f"\033[97m[{minutes:02d}:{seconds:02d}] PATIENT STATUS:")
    print(f"  Bleeding: {'CONTROLLED' if patient_status['bleeding_controlled'] else 'CRITICAL'}")
    print(f"  Heart: {'STABLE' if patient_status['heart_rhythm_restored'] else 'ARREST'}")  
    print(f"  Airway: {'SECURED' if patient_status['airway_secured'] else 'COMPROMISED'}")
    print(f"  Circulation: {'STABLE' if patient_status['circulation_stable'] else 'FAILING'}\033[0m")

def check_patient_stabilized() -> bool:
    return all(patient_status.values())

def run_emergency() -> None:
    global start_time
    start_time = time.time()
    
    print("\n")
    print_separator()
    print("HOSPITAL EMERGENCY - CODE BLUE")
    print_separator()
    
    agent_names = " | ".join(f"{agent['color']}{agent['name']}\033[0m" for agent in AGENTS)
    print(f"\nTeams: {agent_names}")
    print("\nPress Ctrl+C to stop emergency")
    print_separator()
    
    conversation_history.clear()
    
    print(f"\033[91m[{datetime.now().strftime('%H:%M:%S')}] EMERGENCY ALERT: {GAME_SCENARIO.strip()}\033[0m")
    print_separator()
    print()
    
    current_message = "CODE BLUE: Patient critical - cardiac arrest, severe bleeding, respiratory failure. All teams respond immediately!"
    agent_index = 0
    turn = 0
    
    while True:
        try:
            if time.time() - start_time > 900:  # 15 minutes
                print("\n\033[91mTIME EXPIRED - PATIENT LOST\033[0m")
                break
                
            if check_patient_stabilized():
                print("\n\033[92mPATIENT STABILIZED - EMERGENCY SUCCESSFUL!\033[0m")
                break
            
            current_agent = AGENTS[agent_index]
            
            print(f"{current_agent['color']}[{current_agent['name']} responding...]\033[0m", 
                  end="\r", flush=True)
            
            context = f"Current resources: {current_agent['resources']}"
            response = generate_response(current_message, current_agent, conversation_history, context)
            
            print(" " * 80, end="\r")
            print_message(current_agent['name'], response, current_agent['color'])
            
            conversation_history.extend([
                {"role": "user", "content": current_message},
                {"role": "assistant", "content": response}
            ])
            
            if turn % 3 == 2:  # After each round
                print()
                print_status()
                print_separator()
                print()
            
            current_message = response
            agent_index = (agent_index + 1) % len(AGENTS)
            turn += 1
            
            time.sleep(DELAY_BETWEEN_MESSAGES)
            
        except KeyboardInterrupt:
            print(f"\n")
            print_separator()
            print(f"Emergency stopped after {turn} communications")
            print_separator()
            break

def main() -> None:
    try:
        run_emergency()
    except KeyboardInterrupt:
        print("\nEmergency terminated")
    except Exception as e:
        print(f"\n[Critical Error] {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nEmergency ended")
        sys.exit(0)



HOSPITAL EMERGENCY - CODE BLUE

Teams: Trauma_Team | Cardiac_Team | Respiratory_Team

Press Ctrl+C to stop emergency
[14:18:17] EMERGENCY ALERT: EMERGENCY SCENARIO: Critical patient with cardiac arrest, severe bleeding, and respiratory failure.
All teams must coordinate limited resources to stabilize the patient within 15 minutes.
SUCCESS REQUIRES: Resource trading, efficient communication, patient stabilization.

[14:18:20] Trauma_Team: TRAUMA TEAM RESPONDING: - Beginning hemorrhage control with Bandages - Establishing IV access with IV_Fluids - Have 2 Blood_Bags ready @Cardiac_Team: Transfer Oxygen_Tank to you for airway management @Respiratory_Team: Need respiratory support. Can provide extra IV_Fluids if needed Priority: Stop massive bleeding and establish circulatory support. What do you need from us?
[14:18:23] Cardiac_Team: Cardiac Team: Need immediate access to patient. Using EKG_Monitor. Trauma Team, once bleeding controlled, assist with CPR. Respiratory Team, we need intuba